# Alpha–Phi Binary Explorer (v4)

**Patches vs v3**

* `F_soft_exp` – smooth depth divisor with _tunable_ exponent constant `c` (default 0.9) using `2**(c·n/φ)`.
* `iterate_damp` – λ damping applied to _whole core_; Jacobian tame.  Extra grid ranges on `(p,q,lam)`.
* `hybrid_pingpong_soft` – φ‑reciprocal stroke followed by softer compression (`power=comp_p`, `q` adjustable).
* Wider **parameter scan** box: `p ∈ [0.8‥2.2]`, `q ∈ [1‥4]`, `lam ∈ [0.05‥0.25]`, steps = 15.
* CSV log writes only **hits below 1 %** plus summary count.

Raise `mp.mp.dps` if pursuing sub‑ppm error.


In [1]:
import mpmath as mp, csv, matplotlib.pyplot as plt
mp.mp.dps = 250  # bump precision

ALPHA = mp.mpf('7.2973525693e-3')
PHI   = (1 + mp.sqrt(5)) / 2

def fib_word_bits(n):
    a, b = '0', '01'
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

def fib_word_real(n):
    bits = fib_word_bits(n)
    return mp.fsum(int(bit)*mp.mpf(2)**(-(k+1)) for k,bit in enumerate(bits))


## Tunable Smooth Depth Map

In [2]:
# --- Tunable smooth depth map ---
def F_soft_exp(x, n_bits, p=1.8, c=0.9):
    return 1 / (x**p * mp.mpf(2)**(c * n_bits/PHI))

for c in (0.6, 0.9, 1.2):
    print(f'>> c={c}')
    for n in (64,128,256,512,1024,2048):
        y   = F_soft_exp(fib_word_real(n), n, p=1.8, c=c)
        err = abs((y-ALPHA)/ALPHA)
        print(f'  n={n:<4} rel_err={mp.nstr(err,3)}')
    print()


>> c=0.6
  n=64   rel_err=1.0
  n=128  rel_err=1.0
  n=256  rel_err=1.0
  n=512  rel_err=1.0
  n=1024 rel_err=1.0
  n=2048 rel_err=1.0

>> c=0.9
  n=64   rel_err=1.0
  n=128  rel_err=1.0
  n=256  rel_err=1.0
  n=512  rel_err=1.0
  n=1024 rel_err=1.0
  n=2048 rel_err=1.0

>> c=1.2
  n=64   rel_err=1.0
  n=128  rel_err=1.0
  n=256  rel_err=1.0
  n=512  rel_err=1.0
  n=1024 rel_err=1.0
  n=2048 rel_err=1.0



## True Damped Iterate (λ applied to full core)

In [3]:
# --- True damped iterate (wider param space) ---
def iterate_damp(x0, steps=15, p=1.4, q=2, lam=0.15):
    traj=[x0]
    for _ in range(steps):
        core = 1 / (10**q * x0**p)
        x0   = core**lam
        traj.append(x0)
    return traj

seed = fib_word_real(512)
traj = iterate_damp(seed, steps=12, p=1.2, q=2, lam=0.1)
print('k  rel_err')
for k,val in enumerate(traj):
    print(k, mp.nstr(abs((val-ALPHA)/ALPHA), 3))


k  rel_err
0 38.8
1 99.3
2 88.8
3 90.0
4 89.8
5 89.8
6 89.8
7 89.8
8 89.8
9 89.8
10 89.8
11 89.8
12 89.8


## Softer Hybrid Ping‑Pong Map

In [4]:
# --- Softer hybrid ping‑pong ---
def hybrid_pingpong_soft(x0, cycles=6, q=2.5, comp_p=1.4):
    traj=[x0]
    for _ in range(cycles):
        x0 = PHI / x0
        x0 = 1 / (10**q * x0**comp_p)
        traj.append(x0)
    return traj

seed = fib_word_real(128)
traj = hybrid_pingpong_soft(seed, cycles=8, q=2.5, comp_p=1.3)
print('cycle rel_err')
for k,val in enumerate(traj):
    print(k, mp.nstr(abs((val-ALPHA)/ALPHA), 3))


cycle rel_err
0 38.8
1 0.954
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0


## Parameter Scan (wider box, hits <1 %)

In [5]:
# --- Expanded parameter scan ---
hits = []
with open('scan_log_v4.csv','w', newline='') as f:
    w = csv.writer(f); w.writerow(['p','q','lam','bits','rel_err'])
    for bits in (256,512,1024):
        seed = fib_word_real(bits)
        for p in [0.8 + 0.2*i for i in range(8)]:   # 0.8 .. 2.2
            for q in range(1,5):                    # 1..4
                for lam in [0.05*i for i in range(1,6)]:  # 0.05..0.25
                    x = iterate_damp(seed, steps=15, p=p, q=q, lam=lam)[-1]
                    rel = abs((x-ALPHA)/ALPHA)
                    if rel < 0.01:
                        w.writerow([p,q,lam,bits, mp.nstr(rel,12)])
                        hits.append(rel)
print(f'Total hits (<1%): {len(hits)}  CSV -> scan_log_v4.csv')


Total hits (<1%): 0  CSV -> scan_log_v4.csv
